In [1]:
from datetime import datetime as dt

import geopandas as gpd
import numpy as np

from pathlib import Path

import pandas as pd
from pprint import pprint





from src import hls_tools as hls
from src import rio_tools as rt

In [2]:
PARENT_DIR = Path(r"/home/iborlafm/Downloads/Mozambique/hls")

In [3]:
aoi = gpd.read_file(PARENT_DIR / "extent.geojson")
geom = aoi.loc[0, "geometry"]

## List the available files

In [4]:
uri_frame = hls.tabulate_hls_uris(PARENT_DIR.rglob("HLS.*tif"))

## Reshape the frame, rename, dump non-shared bands

In [5]:
both_frame = hls.harmonize_hls_frame(uri_frame)

## Clip and stack the rasters

In [6]:
selected_columns = ["Blue", "Green", "Red", "NIRnarrow", "SWIR1", "SWIR2"]

In [7]:
for observation_name, observation_files in both_frame.iterrows():

    rt.write_raster(
        *rt.clipped_read(observation_files[selected_columns], aoi),
        (PARENT_DIR / f"{observation_name}_bands.tif")
    )

    rt.write_raster(
        *rt.clipped_read(observation_files[["Fmask"]], aoi),
        (PARENT_DIR / f"{observation_name}_Fmask.tif")
    )